In [3]:
import tensorflow as tf
import keras_tuner
import numpy as np


import pandas as pd
import openpyxl # Needed for reading excel
import pathlib

import decomposition
import models
import data
from metrics import smape
from windower import WindowGenerator
import hp_training
import results

In [4]:
cwd = pathlib.Path.cwd()

code_directory = cwd.parents[1]
gonem_directory = code_directory / "notebooks" / "Gonem"
data_directory = code_directory / "data"
hp_directory = code_directory / "hp"
scenario_directory = code_directory / "scenarios"
model_directory = code_directory / 'models'

model = 'ARF' # SS, ARF or ED
product = 'maize' # maize, sunflower or wheat 

data_type = 'in_sample' # in_sample or out_sample ## Does not matter if scenario != 0
scenario = 2 # 0: the normal setting, 1 or 2


model_path =  model_directory / f"{model}_{product}"
checkpoint_path = model_path / f"{model}_{product}"

results_path = model_directory / f"{model}_{product}"

results_path

scenario_files = []
for path in pathlib.Path(scenario_directory).iterdir():
    if path.is_file():
        scenario_files.append(path.name)

In [5]:
if scenario:
    data_type = f"{scenario_files[scenario-1]}"
    print(data_type)
    df = data.get_data(scenario_directory / scenario_files[scenario-1])
else:
    df = data.get_data(directory_path=data_directory, product=product)


df = df.iloc[:-2]
df.describe()

SCENARIO_OILPRICESHOCK_[183, 48, 26, 90, 82]_2023-03-10_02-56-23.xlsx


AVG_TAVG                                                  \
PARTNER_Labels      Brazil      France     Germany     Hungary     Ukraine   
count           212.000000  212.000000  212.000000  212.000000  212.000000   
mean            238.213467  125.120399  103.852255  116.129388   96.215727   
std              24.185228   55.581487   65.415460   79.506412   90.577074   
min             146.692308   20.168095  -29.860742  -50.419892  -98.247057   
25%             228.163849   76.846499   47.985468   42.248987   16.463480   
50%             246.068678  121.139616  101.023015  120.209032   91.978710   
75%             254.833821  177.800263  164.291849  191.053584  183.358356   
max             276.134483  231.747995  229.864177  242.571429  239.800437   

               Corn Price Futures    MAX_TMAX                          \
PARTNER_Labels             Global      Brazil      France     Germany   
count                  212.000000  212.000000  212.000000  212.000000   
mean                   447.205189  345.878032  225.155034  218.462006   
std                    150.596080   24.373066   67.282110   81.540537   
min                    201.750000  282.000000  107.622222   36.388889   
25%                    356.437500  330.875000  163.036330  140.056561   
50%                    389.625000  345.000000  227.637049  225.650735   
75%                    555.750000  360.625000  284.274194  287.716912   
max                    818.250000  401.000000  362.382979  371.823529   

                            ... renewable_energy_consumption_perc_of_total  \
PARTNER_Labels     Hungary  ...                                     Brazil   
count           212.000000  ...                                 212.000000   
mean            238.409827  ...                                  45.961321   
std              86.291146  ...                                   1.909219   
min              43.666667  ...                                  41.710000   
25%             162.625000  ...                                  44.839375   
50%             243.400000  ...                                  46.680417   
75%             317.125000  ...                                  47.570000   
max             392.500000  ...                                  48.920000   

                                                                \
PARTNER_Labels      France     Germany     Hungary     Ukraine   
count           212.000000  212.000000  212.000000  212.000000   
mean             12.858797   13.605825   13.299269    4.511203   
std               2.269171    2.843999    2.701251    2.124186   
min               8.520000    7.280000    7.290000    1.270000   
25%              11.130833   11.072292   12.753125    2.807500   
50%              13.277500   13.938750   13.640000    3.497917   
75%              15.270000   16.316875   15.388750    7.017500   
max              15.530000   17.170000   17.180000    7.440000   

               unemployment_total                                      \
PARTNER_Labels             Brazil      France     Germany     Hungary   
count                  212.000000  212.000000  212.000000  212.000000   
mean                    10.096958    8.980330    5.606981    7.008726   
std                      2.280336    0.864391    2.218326    2.706634   
min                      6.760000    7.390000    3.140000    3.420000   
25%                      8.174167    8.110625    3.664375    4.084583   
50%                      9.502083    9.101250    5.032083    7.355833   
75%                     12.416250    9.801875    7.435208    9.591042   
max                     13.700000   10.350000   11.170000   11.170000   

                            
PARTNER_Labels     Ukraine  
count           212.000000  
mean              8.350071  
std               1.099230  
min               6.350000  
25%               7.462500  
50%               8.503750  
75%               9.272500  
max               9.830000  

[8 rows x 63 columns]

In [6]:
label_columns = ['price']
label_columns = df.columns[df.columns.get_level_values(0).isin(label_columns)].tolist()
label_columns

[('price', 'Brazil'),
 ('price', 'France'),
 ('price', 'Germany'),
 ('price', 'Global'),
 ('price', 'Hungary'),
 ('price', 'Ukraine')]

In [7]:
stl = decomposition.STLDecomposer(labels=label_columns, period=12)
log = decomposition.Logger(labels=label_columns)
std = decomposition.Standardizer()

preproc = decomposition.Processor().add(stl).add(log).add(std)

In [8]:
width = 24
label_width = 6
shift = 6

if data_type == 'in_sample':
    test_begin = None
else:
    test_begin = 0.
    
window = WindowGenerator(input_width=width, label_width=label_width, shift=shift, data=df, 
                    # train_begin=0, train_end=.9, val_begin=None, val_end=.96,
                    train_begin=0., train_end=.97, val_begin=None, val_end=None,
                    # train_begin=0, train_end=.5, val_begin=None, val_end=.8,
                    test_begin=test_begin, test_end=1., connect=True, remove_labels=True, label_columns=label_columns)
window.preprocess(preproc)
window

Total window size: 30
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29]
Label column name(s): [('price', 'Brazil'), ('price', 'France'), ('price', 'Germany'), ('price', 'Global'), ('price', 'Hungary'), ('price', 'Ukraine')]

In [9]:
label_std = decomposition.Standardizer(mean=std.mean[window.label_columns], std=std.std[window.label_columns])
label_log = decomposition.Logger(label_indices=range(len(window.label_columns)))
postproc = decomposition.Processor().add(label_std).add(label_log)
window.add_label_postprocess(postproc)

In [12]:
inputs = tf.convert_to_tensor(np.load(model_path /  f"{product}_inputs_{data_type}.npy"))
labels = tf.convert_to_tensor(np.load(model_path /  f"{product}_labels_{data_type}.npy"))
weights = tf.convert_to_tensor(np.load(model_path / f"{product}_weights_{data_type}.npy"))
predictions = tf.convert_to_tensor(np.load(model_path / f"{product}_predictions_{data_type}.npy"))
mcds = tf.convert_to_tensor(np.load(model_path / f"{product}_mcd_predictions_{data_type}.npy"))

In [11]:
inputs.shape, labels.shape, weights.shape, predictions.shape, mcds.shape

(TensorShape([183, 24, 75]),
 TensorShape([183, 6, 6]),
 TensorShape([1, 75]),
 TensorShape([183, 6, 6]),
 TensorShape([100, 183, 6, 6]))

In [81]:
results.weight_results(weights[0], window.train_df.columns)

,country,price_trend,price_seasonal,price_residual,AVG_TAVG,Corn Price Futures,MAX_TMAX,MIN_TMIN,OBS_VALUE_QUANTITY_TON,OIL_PRICE $/bbl,agricultural_land,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,gdp_current_us,population_total,renewable_energy_consumption_perc_of_total,unemployment_total,price
0,Brazil,0.494678,0.739065,0.500782,0.254973,NaN,0.988430,0.168805,0.529142,NaN,0.020744,0.507422,0.107799,0.122484,0.079451,0.437595,0.042050,NaN
1,France,0.993519,0.720823,0.991954,0.413029,NaN,0.035834,0.150684,0.116774,NaN,0.175720,0.073418,0.244538,0.526178,0.305757,0.033009,0.084799,NaN
2,Germany,0.481186,0.879754,0.846405,0.062893,NaN,0.558997,0.544435,0.811979,NaN,0.975717,0.421624,0.045476,0.941260,0.026491,0.005236,0.481805,NaN
3,Global,0.913110,0.940519,0.987832,NaN,0.455777,NaN,NaN,NaN,0.510973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hungary,0.075148,0.995326,0.310927,0.334205,NaN,0.572407,0.007502,0.002124,NaN,0.021501,0.220700,0.783732,0.487172,0.002497,0.442570,0.506175,NaN
5,Ukraine,0.501829,0.404425,0.505124,0.059433,NaN,0.732472,0.559622,0.723768,NaN,0.587294,0.317992,0.052708,0.991868,0.525882,0.061814,0.993148,NaN


In [82]:
intervals = results.forecast_interval(mcds, 0.1)

In [72]:
import metrics
for i in range(6):
    print(metrics.smape(predictions[:, 5, i], labels[:, 5, i])/2)

tf.Tensor(14.429292, shape=(), dtype=float32)
tf.Tensor(11.13741, shape=(), dtype=float32)
tf.Tensor(26.910423, shape=(), dtype=float32)
tf.Tensor(12.702807, shape=(), dtype=float32)
tf.Tensor(7.5614967, shape=(), dtype=float32)
tf.Tensor(14.304833, shape=(), dtype=float32)
